In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from multiprocessing import Pool, cpu_count    

In [ ]:
%matplotlib inline

# 1. Data Preprocessing

In [ ]:
img_height = 128
img_width  = 128

In [ ]:
inv_label_map = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [ ]:
df_train = pd.read_csv('../input/train.csv')
Y = df_train.iloc[:,1:].values
names = df_train['image_name']

i = 0
X = np.empty((names.shape[0], img_height, img_width, 3), dtype=np.float16)
for f in tqdm(names.values, miniters=1000):
    img = cv2.imread('../input/train-jpg/{}.jpg'.format(f))
    if img_height != img.shape[0]:
        img = cv2.resize(img, (img_height, img_width))
    X[i,:,:,:] = np.array(img, np.float16)
    i += 1
X = X / 255.

#deprecated parallel reading because exceed memory when passing data back
'''
def get_images(names):
    i = 0
    X = np.empty((names.shape[0], img_height, img_width, 3), dtype=np.float16)
    for f in tqdm(names.values, miniters=1000):
        img = cv2.imread('../input/train-jpg/{}.jpg'.format(f))
        if img_height != img.shape[0]:
            img = cv2.resize(img, (img_height, img_width))
        X[i,:,:,:] = np.array(img, np.float16)
        i += 1
    return X / 255.

#multiply cpu_count if cannot fit memory
pool = Pool(cpu_count())
X = np.concatenate(pool.map(
    get_images, 
    np.array_split(df_train['image_name'], cpu_count())
))
pool.close()
pool.join()'''
print(X.shape)

# 2. Model Training

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def fbeta(y_true, y_pred):
    beta = 2
    threshold_shift = -0.3

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

In [ ]:
base_model = VGG16(input_shape=(img_height,img_width,3), include_top=False)
for layer in base_model.layers[:10]: 
    layer.trainable = False

new_model = base_model.output
new_model = Flatten()(new_model)
new_model = Dense(4096, activation='relu')(new_model)
new_model = Dropout(0.5)(new_model)
new_model = Dense(4096, activation='relu')(new_model)
new_model = Dropout(0.5)(new_model)
new_model = Dense(17, activation='sigmoid')(new_model)

model = Model(inputs=base_model.input, outputs=new_model)

In [ ]:
epochs_arr = [20, 5, 5]
learn_rates = [0.001, 0.0001, 0.00001]
kfold_weights_path = os.path.join('', 'weights.h5')

for learn_rate, epochs in zip(learn_rates, epochs_arr):
    opt  = Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer=opt,
                  metrics=['accuracy', fbeta])
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=2, verbose=2),
        ModelCheckpoint(kfold_weights_path, monitor='val_loss', 
                        save_best_only=True, verbose=2)
    ]
    
    #deprecated generator because exceed memory
    '''model.fit_generator(train_generator.flow(x_train, y_train, batch_size=128),
          steps_per_epoch=len(x_train) / 128,
          epochs=epochs,
          verbose=1,
          workers=3,
          validation_data=(x_valid, y_valid),
          callbacks=callbacks)'''

    model.fit(x = x_train, y= y_train, validation_data=(x_valid, y_valid),
      batch_size=128,verbose=1, epochs=epochs,callbacks=callbacks,shuffle=True)

In [ ]:
#save!
model.save_weights('final.h5')

In [ ]:
'''opt  = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer=opt,
                  metrics=['accuracy', fbeta])
model.load_weights('final.h5')'''

In [ ]:
kfold_weights_path = os.path.join('', 'weights.h5')
if os.path.isfile(kfold_weights_path):
    model.load_weights(kfold_weights_path)

# 3. Model Evaluation

In [ ]:
from sklearn.metrics import fbeta_score, accuracy_score

In [ ]:
p_valid = model.predict(x_valid, batch_size=128, verbose=1)

In [ ]:
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

In [ ]:
#save f2 score for stage 2 weighted
scores = fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average=None)
print('F2 test scores per tag:')
for label, score in [(inv_label_map[l], scores[l]) for l in scores.argsort()[::-1]]:
    print(label, ': ', score)
pd.DataFrame([scores]).to_csv('f2.csv', index=False)

In [ ]:
for i in range(17):
    print(inv_label_map[i], '\t:', accuracy_score(y_valid[:,i], p_valid[:,i]>0.2))

In [ ]:
#predict train data for stage 2
p_train = model.predict(X, batch_size=128, verbose=1)
pd.DataFrame(p_train).to_csv('train.csv', index=False, float_format='%.3f')

# 3. Make Prediction

In [ ]:
df_submission = pd.read_csv('../input/sample_submission_v2.csv')

def get_images(names):
    i = 0
    X = np.empty((names.shape[0], img_height, img_width, 3), dtype=np.float16)
    for f in tqdm(names.values, miniters=1000):
        img = cv2.imread('../input/test-jpg/{}.jpg'.format(f))
        if img_height != img.shape[0]:
            img = cv2.resize(img, (img_height, img_width))
        X[i,:,:,:] = np.array(img, np.float16)
        i += 1
    return X / 255.

pool = Pool(cpu_count())
X_submission = np.concatenate(pool.map(
    get_images, 
    np.array_split(df_submission['image_name'], cpu_count())
))
pool.close()
pool.join()
print(X_submission.shape)

In [ ]:
predict = model.predict(X_submission, batch_size = 128, verbose=1)

In [ ]:
#use raw wihout threshold!
result = pd.DataFrame(np.array(predict) > 0.2)
preds = []
sorted_tags = pd.Series(inv_label_map)

for i in tqdm(range(result.shape[0]), miniters=1000):
    preds.append(' '.join(list(
        sorted_tags[np.where(result.loc[i] == 1)[0]]
    )))

In [ ]:
df_submission['tags'] = preds
df_submission.to_csv('test.csv', index=False)